In [30]:
import os
import pandas as pd
import numpy
import matplotlib.pyplot as plt
from IPython.display import Image
%matplotlib inline
plt.style.use('ggplot')

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 20)
pd.set_option('display.notebook_repr_html', True)

# Loading dataset
df = pd.read_csv(os.path.join('..', 'dataset', 'dataset-13-walget-train.csv'))
# df = df.set_index('business_id')


In [31]:
df.describe

<bound method DataFrame.describe of     AccountHolderImpliedGender AccountHolderAddress  \
0                       Female                 Home   
1                       Female               PO Box   
2                         Male            Apartment   
3                         Male                 Home   
4                       Female               PO Box   
5                       Female                 Home   
6                         Male            Apartment   
7                         Male            Apartment   
8                       Female            Apartment   
9                       Female            Apartment   
..                         ...                  ...   
590                       Male            Apartment   
591                       Male                 Home   
592                       Male            Apartment   
593                       Male                 Home   
594                       Male                 Home   
595                       Mal

Some Notes
Since the model is trying to classify between pregnant and not pregnant, we are planning to use a logistic regresssion
In terms of features, the features that we thought were most relevant are "RecentlyPurchasedPregnancyTest", "RecentlyPurchasedFolicAcidSupplements", "RecentlyPurchasedPrenatalVitamins", "RecentlyPurchasedMaternityClothing"

In [32]:
df.columns

Index([u'AccountHolderImpliedGender', u'AccountHolderAddress',
       u'RecentlyPurchasedPregnancyTest', u'RecentlyPurchasedBirthControl',
       u'RecentlyPurchasedFeminineHygieneProducts',
       u'RecentlyPurchasedFolicAcidSupplements',
       u'RecentlyPurchasedPrenatalVitamins',
       u'RecentlyPurchasedPrenatalYogaDVD', u'RecentlyPurchasedBodyPillow',
       u'RecentlyPurchasedGingerAle', u'RecentlyPurchasedSeaBands',
       u'PurchasedCigarettesRegularlyUntilRecentlyThenStopped',
       u'RecentlyPurchasedCigarettes',
       u'RecentlyPurchasedSmokingCessationProducts',
       u'PurchasedWineRegularlyUntilRecentlyThenStopped',
       u'RecentlyPurchasedWine', u'RecentlyPurchasedMaternityClothing',
       u'IsPregnant'],
      dtype='object')

In [33]:
df2 = df[['RecentlyPurchasedPregnancyTest', 'RecentlyPurchasedFolicAcidSupplements', 'RecentlyPurchasedPrenatalVitamins', 'RecentlyPurchasedMaternityClothing', 'IsPregnant']]
df2

,RecentlyPurchasedPregnancyTest,RecentlyPurchasedFolicAcidSupplements,RecentlyPurchasedPrenatalVitamins,RecentlyPurchasedMaternityClothing,IsPregnant
0,False,False,True,False,True
1,False,False,False,False,True
2,False,False,False,False,False
3,False,False,True,False,True
4,False,False,False,False,False
5,False,False,False,False,False
6,False,False,True,False,False
7,False,False,True,False,True
8,True,True,True,False,True
9,False,False,False,False,True


In [34]:
# Drop all nulls since is a small number
df2 = df2.dropna()
len(df2)

584

In [35]:
df2=df2*1
df2.head()

,RecentlyPurchasedPregnancyTest,RecentlyPurchasedFolicAcidSupplements,RecentlyPurchasedPrenatalVitamins,RecentlyPurchasedMaternityClothing,IsPregnant
0,0,0,1,0,1
1,0,0,0,0,1
2,0,0,0,0,0
3,0,0,1,0,1
4,0,0,0,0,0


In [42]:
df2.groupby("IsPregnant").s

,RecentlyPurchasedPregnancyTest,RecentlyPurchasedFolicAcidSupplements,RecentlyPurchasedPrenatalVitamins,RecentlyPurchasedMaternityClothing
IsPregnant,,,,
0,4,0,13,12
1,41,65,69,65


KeyError: 'IsPregnant'